In [1]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
import os
import random
import numpy as np
import threading
from PIL import Image
from skimage import io, color
from skimage.transform import rescale, resize, downscale_local_mean
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.models import Model, load_model
from keras.optimizers import *
from keras.layers import *
from keras.layers.core import Lambda
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from slice import Slice
from colorizer import create_colorizer
from discriminator import create_discriminator


# We need only the conv part of VGG
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
image_mean = (103.939, 116.779, 123.68)
greyscale_image_mean = np.mean(image_mean)

Using TensorFlow backend.


In [2]:
def make_trainable(layers, is_trainable):
    for l in layers:
        l.trainable = is_trainable
        
def create_GAN(generator, discriminator):
    generator_trainable_layers = [layer for layer in generator.layers if layer.trainable]
    discriminator_trainable_layers = [layer for layer in discriminator.layers if layer.trainable]

    network_input = Input(shape=(224, 224, 3))
    generator_output = generator(network_input)

    lightness = Slice(slice(0, 1), axis=3) (network_input)
    lab = concatenate([lightness, generator_output])
    network_output = discriminator(lab)

    GAN = Model(network_input, network_output)
    return GAN, generator_trainable_layers, discriminator_trainable_layers

In [3]:
colorizer, vgg = create_colorizer()
discriminator = create_discriminator()
gan, generator_trainable_layers, discriminator_trainable_layers = create_GAN(colorizer, discriminator)

discriminator.compile(loss='binary_crossentropy', optimizer='adam')
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [4]:
img = load_img('./dog.jpg', target_size=(224, 224))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

# res_first = first.predict([x], 1, verbose=1)
res_second = colorizer.predict([x], 1, verbose=1)

# print(res_first[0].shape)
print(res_second[0].shape)

1/1 [==============================] - 4s
(224, 224, 2)


# Train

In [10]:
import random
import numpy as np
import threading

class BaseBatchGenerator:
    
    def __init__(self, image_paths, batch_size, image_height, image_width, shuffle = True):
        self.image_paths = image_paths
        self.batch_size = batch_size
        self.image_height = image_height
        self.image_width = image_width
        self.lock = threading.Lock()
        self.index = 0
        if shuffle:
            random.shuffle(self.image_paths)

    def next(self):
        with self.lock:
            batch_features = []
            batch_labels   = []

            for i in range(self.batch_size):
                path = self.image_paths[self.index]
                x, y = generate_one(path)

                self.index += 1
                if self.index >= len(self.image_paths):
                    self.index = 0
            
            return np.array(batch_features), np.array(batch_labels)
    
    def generate_one(path):
        raise NotImplementedError("Please Implement this method")

In [5]:
class BatchGenerator:

    def __init__(self, image_paths, batch_size, image_height, image_width, shuffle = True):
        self.image_paths = image_paths
        self.batch_size = batch_size
        self.image_height = image_height
        self.image_width = image_width
        self.generate_for_discriminator = True
        self.lock = threading.Lock()
        self.index = 0
        if shuffle:
            random.shuffle(self.image_paths)

    def next(self, is_real):
        with self.lock:
            batch_features = np.zeros((self.batch_size, self.image_height, self.image_width, 3))
            batch_labels   = np.zeros((self.batch_size))

            for i in range(self.batch_size):
                path = self.image_paths[self.index]
                
                if self.generate_for_discriminator:
                    image_gray = load_img(path, grayscale=True, target_size=(self.image_height, self.image_width))
                    batch_features[i,:,:,0] = image_gray - greyscale_image_mean
                    batch_features[i,:,:,1] = image_gray - greyscale_image_mean
                    batch_features[i,:,:,2] = image_gray - greyscale_image_mean
                    batch_labels[i] = is_real
                else:
                    image = io.imread(path)
                    image = resize(image, output_shape=(self.image_height, self.image_width, 3))
                    image_gray = load_img(path, grayscale=True, target_size=(self.image_height, self.image_width))
                    batch_features[i] = color.rgb2lab(image)
                    batch_features[i,:,:,0] = image_gray - greyscale_image_mean
                    batch_labels[i] = is_real

                self.index += 1
                if self.index >= len(self.image_paths):
                    self.index = 0
            
            return batch_features, batch_labels

In [6]:
# Define paths for train / validation / test datasets
train_root = '../coco/train2014/'
valid_root = '../coco/val2014/'
test_root = '../coco/test2014/'

train_image_paths = [train_root + item for item in os.listdir(train_root) if item.endswith('.jpg') ]
valid_image_paths = [valid_root + item for item in os.listdir(valid_root) if item.endswith('.jpg') ]
test_image_paths = [test_root + item for item in os.listdir(test_root) if item.endswith('.jpg') ]

print( 'Train:', len(train_image_paths) )
print( 'Validation:', len(valid_image_paths) )
print( 'Test:', len(test_image_paths) )

Train: 82783
Validation: 40504
Test: 40775


In [7]:
batch_size = 37
train_data_generator = BatchGenerator(image_paths=train_image_paths, batch_size=batch_size, image_height=224, image_width=224)
valid_data_generator = BatchGenerator(image_paths=valid_image_paths, batch_size=batch_size, image_height=224, image_width=224)

print('There are', len(train_image_paths) / batch_size, 'steps per one train epoch for one complete cycle')
print('There are', len(valid_image_paths) / batch_size, 'steps per one validation epoch for one complete cycle')

checkpoint_path="../checkpoints/model-{epoch:03d}-{loss:.4f}.model"
model_saver = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', save_best_only=True, mode='min')
# tensorboard = TensorBoard(log_dir='/tmp/coloring/')


epochs = 77
steps_per_epoch = 500
validation_steps = 33
# callbacks = [tensorboard, model_saver]

There are 2237 steps per one train epoch for one complete cycle
There are 1094 steps per one validation epoch for one complete cycle


In [8]:
from tqdm import tqdm
from tqdm import trange

for epoch in range(epochs):
    
    steps = [0, 1, 2] * steps_per_epoch
    t = trange( len(steps) )
    for i in t:
        step = steps[i]
        
        # Make discriminator better on predicting real images
        if step == 0:
            train_data_generator.generate_for_discriminator = True
            net = discriminator                                   # Propagate the data only through discriminator
            make_trainable(discriminator_trainable_layers, True)  # make discriminator layers trainable
            make_trainable(generator_trainable_layers, False)     # we don't touch the generator
            batch = train_data_generator.next(is_real=True)       # Images are real
                
        # Make discriminator better on detecting fake images
        elif step == 1:
            train_data_generator.generate_for_discriminator = False
            net = gan                                             # Propagate the data though the whole gan
            make_trainable(discriminator_trainable_layers, True)  # Train the discriminator
            make_trainable(generator_trainable_layers, False)     # But don't touch the generator as it's not his turn
            batch = train_data_generator.next(is_real=False)      # Images are fake
        
        # Make colorizer better in fooling the discriminator
        else:
            train_data_generator.generate_for_discriminator = False
            net = gan                                             # Propagate the data through the whole gan
            make_trainable(discriminator_trainable_layers, False) # Don't train the discriminator when fooling it
            make_trainable(generator_trainable_layers, True)      # Train only the generator
            batch = train_data_generator.next(is_real=True)       # Lets fool the discriminator

        
        x, y = batch
        loss = net.train_on_batch(x, y)
        
        # Display progress
        t.set_description('Train batch %i' % i)
        t.set_postfix(loss=loss)

Train batch 0:   0%|          | 1/1500 [00:27<11:36:49, 27.89s/it, loss=0.729]/home/martin/.miniconda2/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Train batch 370:  25%|██▍       | 371/1500 [47:26<2:23:43,  7.64s/it, loss=15.9]    

KeyboardInterrupt: 

# Evaluate

In [ ]:
model = load_model('../checkpoints/model-000-730.5353.model')

In [ ]:
path = './puma.jpg' # random.choice(test_image_paths)

rgb = io.imread(path)
rgb = resize(rgb, output_shape=(224, 224, 3))
image_gray = load_img(path, grayscale=True, target_size=(224, 224))
lab = color.rgb2lab(rgb)

inp = np.zeros((224, 224, 3))
inp[:,:,0] = image_gray - greyscale_image_mean
inp[:,:,1] = image_gray - greyscale_image_mean
inp[:,:,2] = image_gray - greyscale_image_mean

In [ ]:
out = model.predict(np.array([inp]), batch_size=1)[0]

img = np.zeros((224, 224, 3))
img[:,:,0] = lab[...,0]
img[:,:,1:] = out

plt.imshow(color.lab2rgb(img))

# Experiments